In [137]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
import unidecode


In [138]:
def cleanStr(i):
    if len(i.split(",")) > 1:
        i = int(i.split(",")[0])
    if "§" in str(i):
        i=i.split("§")[0]
    if "-" in str(i):
        i=i.split("-")[0]
    if "." in str(i):
        i=i.split(".")[0]
    return int(i)

def returnHour(i):
    try:
        i = i.replace(":","")[:2]
    except:
        i = 99
    return i

def returnMinute(i):
    try:
        i = i.replace(":","")[2:]
    except:
        i = 99
    return i

def limpaCidade(x):
    x = str(x).rstrip()
    x = x.replace("S.","SÃO ")
    x = x.replace("EMBU-GUACU", "EMBU GUACU")
    x = x.replace("SÃO ANDRE","SANTO ANDRE")
    x = x.replace("SÃO ISABEL","SANTA ISABEL")
    x = unidecode.unidecode(x)
    return x


In [139]:
df_0 = pd.read_csv("archive\BO_2007_1.csv")
df_1 = pd.read_csv("archive\BO_2007_2.csv")

d:\Ciencias de Dados\Projetos\Concluidos\Ciencias-de-dados\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (17,18,21,26) have mixed types.Specify dtype option on import or set low_memory=False.

d:\Ciencias de Dados\Projetos\Concluidos\Ciencias-de-dados\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (24,26) have mixed types.Specify dtype option on import or set low_memory=False.



In [140]:
df = pd.concat([df_0,df_1])

In [141]:
df = df[df["ANO_BO"] == 2007]
df["RUBRICA_x"] = df["RUBRICA"].apply(lambda x: x.split("(")[1][4:].replace(")",""))
df["RUBRICA_final"] = df["RUBRICA_x"].apply(lambda x: cleanStr(x))
df["H_OCORRENCIA_BO"] = df["HORA_OCORRENCIA_BO"].apply(lambda x: returnHour(x))
df["M_OCORRENCIA_BO"] = df["HORA_OCORRENCIA_BO"].apply(lambda x: returnMinute(x))
df["DT_OCORRENCIA_BO"] = pd.to_datetime(df["DATA_OCORRENCIA_BO"])
df["DESCR_TIPO_PESSOA_clean"] = df["DESCR_TIPO_PESSOA"].apply(lambda x: str(x).rstrip())
df["MES_OCORRENCIA_BO"] = df["DT_OCORRENCIA_BO"].apply(lambda x:x.month)
df["LOGRADOURO"] = df["LOGRADOURO"].apply(lambda x: str(x).rstrip())
df["LOGRADOURO"] = df["LOGRADOURO"].apply(lambda x: str(x).lstrip())
df["CIDADE"] = df["CIDADE"].apply(lambda x: limpaCidade(x))
df["SEXO_PESSOA"] = df["SEXO_PESSOA"].apply(lambda x: str(x).rstrip())
df["COR"] = df["COR"].apply(lambda x: str(x).rstrip())
df["NOME_SECCIONAL"] = df["NOME_SECCIONAL"].apply(lambda x: str(x).rstrip())

descricao_le = LabelEncoder()
df["DESCR_TIPO_PESSOA_le"] = descricao_le.fit_transform(df["DESCR_TIPO_PESSOA"])
logradouro_le = LabelEncoder()
df["LOGRADOURO_le"] = logradouro_le.fit_transform(df["LOGRADOURO"])
city_le = LabelEncoder()
df["CIDADE_le"] = city_le.fit_transform(df["CIDADE"])
sexo_le = LabelEncoder()
df["SEXO_PESSOA_le"] = sexo_le.fit_transform(df["SEXO_PESSOA"])
cor_le = LabelEncoder()
df["COR_le"] = cor_le.fit_transform(df["COR"])

secc_le = LabelEncoder()
df["NOME_SECCIONAL_le"] = secc_le.fit_transform(df["NOME_SECCIONAL"])

In [142]:
df_clean = df[['NUM_BO', 'ANO_BO', 'MES_OCORRENCIA_BO', 'NOME_SECCIONAL_le', 'ID_DELEGACIA', 'DT_OCORRENCIA_BO', 
    'H_OCORRENCIA_BO',"M_OCORRENCIA_BO", 'RUBRICA_final', 'CIDADE_le','CIDADE', 'LOGRADOURO_le',
    'DESCR_TIPO_PESSOA_le', 'SEXO_PESSOA_le', 'COR_le']]

In [143]:
bo_count = df_clean.groupby("MES_OCORRENCIA_BO").count()["NUM_BO"].reset_index()
fig = px.bar(bo_count, x='MES_OCORRENCIA_BO', y='NUM_BO')
fig.show()

In [144]:
bo_count_city = df_clean.groupby("CIDADE").count()["NUM_BO"].reset_index()
bo_count_city_more = bo_count_city.sort_values("NUM_BO",ascending=False).reset_index()[:10]
bo_count_city_less = bo_count_city.sort_values("NUM_BO").reset_index()[-10:]
fig = px.bar(bo_count_city_more, x='CIDADE', y='NUM_BO')
fig.show()